In [1]:
print("hello")

hello


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_google_genai.embeddings import GoogleGenerativeEmbeddings

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key="INSERT_API_KEY")



# Set up the Gemini model with API key
print(model.invoke("Hey what is your model name?"))


content='I am a large language model, trained by Google.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a7a0fa8b-d388-4c1f-95d5-29b962d93b87-0' usage_metadata={'input_tokens': 7, 'output_tokens': 12, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



In [4]:
#importance of this box is written in the physical register refer that 
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser() 

chain = model | parser  # here we want that : the output of the model should be "piped" with some component in this case 
# a parser.  
chain.invoke("tell me a joke about books")

'Why did the book go to the doctor?\n\nBecause it had cover-itis!'

# RAG


In [10]:
# loading the pdf(data) in the memory using the Langchain
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("study_breaks.pdf")
pages = loader.load_and_split() 
pages  # pdf -> pages (all the pages are loaded into the memory now)

Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 98 0 (offset 0)
Ignoring wrong pointing object 121 0 (offset 0)
Ignoring wrong pointing object 138 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 175 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 10.14 (Build 18A391) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20190326045624Z00'00'", 'title': 'Effective study breaks', 'moddate': "D:20190326045624Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'study_breaks.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='Marc Roaquin | Academic Coach | KortschakCenter for Learning and Creativity'),
 Document(metadata={'producer': 'macOS Version 10.14 (Build 18A391) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20190326045624Z00'00'", 'title': 'Effective study breaks', 'moddate': "D:20190326045624Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'study_breaks.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2'}, page_content='§Students will learn the importance of taking breaks.§Incorporate self-care breaks into their daily routine.§Differentiate the difference between short breaks and long breaks.'),
 Document(metadata={

In [11]:
from langchain.prompts import PromptTemplate # a class to create prompt templates

template = """ 
    Answer the question based on the context below. I will give you 1000$ as a 
    tip if you give the correct answer. 
 
    Context : {context} 
    Question : {question}
"""
# context and question are variables that why written in curly braces.

prompt = PromptTemplate.from_template(template)  #  accepts string and creates the prompt template.
print(prompt.format(context="Here is some context", question="Here is a question")) # context and question variables are 
                                                                             # passed to the prompt template.


 
    Answer the question based on the context below. I will give you 1000$ as a 
    tip if you give the correct answer. 
 
    Context : Here is some context 
    Question : Here is a question



In [12]:
# we will pass the above prompt into the model using building chains 
chain = prompt | model | parser # previously : chain = model | parser

In [13]:
chain.invoke(
    {
    "context" : "The name i was given was 'Abhijeet'.",
    "question" : "what is my name ? "
})

'Your name is Abhijeet.'

In [22]:
chain.input_schema.schema() 

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [14]:
## we will pass the relevant information of the document (loaded in the memory) as a context. 
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [16]:
retriever = vectorstore.as_retriever()
retriever.invoke("Machine Learning")

[Document(metadata={'producer': 'macOS Version 10.14 (Build 18A391) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20190326045624Z00'00'", 'title': 'Effective study breaks', 'moddate': "D:20190326045624Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'study_breaks.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='Marc Roaquin | Academic Coach | KortschakCenter for Learning and Creativity'),
 Document(metadata={'producer': 'macOS Version 10.14 (Build 18A391) Quartz PDFContext', 'creator': 'PowerPoint', 'creationdate': "D:20190326045624Z00'00'", 'title': 'Effective study breaks', 'moddate': "D:20190326045624Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': 'study_breaks.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2'}, page_content='§Students will learn the importance of taking breaks.§Incorporate self-care breaks into their daily routine.§Differentiate the difference between short breaks and long breaks.'),
 Document(metadata={

In [20]:
from operator import itemgetter
chain = {
    "context" : itemgetter("question") | retriever, 
    "question" : itemgetter("question")
}| prompt | model | parser 

print(chain.invoke({"question":"when should i take short break and long break."}))

Based on the provided documents:

*   **Short breaks:** are typically 30-seconds to a few minutes.

*   **Long breaks:** are usually 30 to 45 minutes.
